# CNN untuk Klasifikasi Penyakit Tanaman Kentang

Proyek ini menggunakan Convolutional Neural Network (CNN) murni berbasis PyTorch untuk mengklasifikasikan penyakit pada daun kentang ke dalam 3 kategori: Healthy, Early Blight, dan Late Blight.

## 1. Import Libraries dan Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set seed untuk reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\seaborn\_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.4)
  from scipy.stats import gaussian_kde


Using device: cpu


## 2. Data Loading dengan Augmentasi

In [ ]:
def load_datasets(data_dir, batch_size=32, val_split=0.2, test_split=0.15):
    """
    Load dataset dengan augmentasi on-the-fly untuk training.
    
    Args:
        data_dir: Path ke folder dataset
        batch_size: Ukuran batch
        val_split: Proporsi data untuk validasi
        test_split: Proporsi data untuk testing (dari sisa setelah train)
    
    Returns:
        train_loader, val_loader, test_loader, test_dataset, num_classes
    """
    # Transformasi untuk training (dengan augmentasi)
    train_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),  # Konversi ke [0,1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Transformasi untuk validation dan test (tanpa augmentasi)
    eval_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load seluruh dataset untuk training/validation/test
    full_dataset = datasets.ImageFolder(
        root=Path(data_dir),
        transform=train_transform
    )
    
    # Split menjadi train, validation, dan test
    total_size = len(full_dataset)
    test_size = int(test_split * total_size)
    train_val_size = total_size - test_size
    val_size = int(val_split * train_val_size)
    train_size = train_val_size - val_size
    
    # Split dataset
    train_dataset, val_test_dataset = random_split(
        full_dataset, 
        [train_size + val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_dataset, val_dataset_temp = random_split(
        train_dataset, 
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    # Buat dataset dengan transform tanpa augmentasi untuk val dan test
    full_dataset_eval = datasets.ImageFolder(
        root=Path(data_dir),
        transform=eval_transform
    )
    
    # Gunakan indices yang sama untuk validasi dan test
    val_indices = [train_dataset.indices[i] for i in val_dataset_temp.indices]
    test_indices = val_test_dataset.indices
    
    val_dataset = torch.utils.data.Subset(full_dataset_eval, val_indices)
    test_dataset = torch.utils.data.Subset(full_dataset_eval, test_indices)
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                             shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                           shuffle=False, num_workers=0, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                            shuffle=False, num_workers=0, pin_memory=True)
    
    num_classes = len(full_dataset.classes)
    
    print(f"Dataset split:")
    print(f"  Total images: {total_size}")
    print(f"  Train set: {len(train_dataset)} images ({len(train_dataset)/total_size*100:.1f}%)")
    print(f"  Validation set: {len(val_dataset)} images ({len(val_dataset)/total_size*100:.1f}%)")
    print(f"  Test set: {len(test_dataset)} images ({len(test_dataset)/total_size*100:.1f}%)")
    print(f"\nClasses ({num_classes}): {full_dataset.classes}")
    
    return train_loader, val_loader, test_loader, full_dataset_eval, num_classes

# Load data - gunakan relative path
data_dir = "../dataset"
train_loader, val_loader, test_loader, test_dataset, num_classes = load_datasets(
    data_dir, batch_size=32, val_split=0.2, test_split=0.15
)

Train set: 1721 images
Validation set: 431 images
Test set: 2152 images
Classes (3): ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


## 3. Arsitektur CNN untuk Klasifikasi

In [3]:
class PotatoCNN(nn.Module):
    """
    CNN murni untuk klasifikasi penyakit kentang.
    Arsitektur:
    - 4 Convolutional blocks dengan BatchNorm dan MaxPooling
    - Global Average Pooling
    - Fully connected layers dengan Dropout
    """
    def __init__(self, num_classes=3):
        super(PotatoCNN, self).__init__()
        
        # Convolutional layers
        self.conv_blocks = nn.Sequential(
            # Block 1: 3 -> 32 (128x128 -> 64x64)
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.1),
            
            # Block 2: 32 -> 64 (64x64 -> 32x32)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),
            
            # Block 3: 64 -> 128 (32x32 -> 16x16)
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.3),
            
            # Block 4: 128 -> 256 (16x16 -> 8x8)
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.4),
        )
        
        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        # Convolutional feature extraction
        x = self.conv_blocks(x)
        
        # Global average pooling
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        
        # Classification
        x = self.classifier(x)
        return x

# Inisialisasi model
model = PotatoCNN(num_classes=num_classes).to(device)
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

PotatoCNN(
  (conv_blocks): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.1, inplace=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

## 4. Training Setup (Loss, Optimizer, Scheduler, Early Stopping)

In [ ]:
class EarlyStopping:
    """Early stopping untuk menghentikan training jika val_loss tidak membaik"""
    def __init__(self, patience=7, min_delta=0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

class ModelCheckpoint:
    """Simpan model terbaik berdasarkan validation loss"""
    def __init__(self, filepath, verbose=True):
        self.filepath = filepath
        self.verbose = verbose
        self.best_loss = float('inf')
        
    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'val_loss': val_loss
            }, self.filepath)
            if self.verbose:
                print(f"Model saved with val_loss: {val_loss:.6f}")

# Setup training components
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=5, factor=0.5
)
early_stopping = EarlyStopping(patience=10, verbose=True)
model_checkpoint = ModelCheckpoint(
    filepath="../model/best_cnn_model.pth",
    verbose=True
)

print("Training components initialized:")
print(f"- Loss: CrossEntropyLoss")
print(f"- Optimizer: Adam (lr=1e-3, weight_decay=1e-4)")
print(f"- Scheduler: ReduceLROnPlateau (patience=5, factor=0.5)")
print(f"- Early Stopping: patience=10")
print(f"- Model Checkpoint: best_cnn_model.pth")

Training components initialized:
- Loss: CrossEntropyLoss
- Optimizer: Adam (lr=1e-3, weight_decay=1e-4)
- Scheduler: ReduceLROnPlateau (patience=5, factor=0.5)
- Early Stopping: patience=10
- Model Checkpoint: best_cnn_model.pth


## 5. Fungsi Training dan Validation

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train untuk satu epoch"""
    model.train()
    running_loss = 0.0
    running_corrects = 0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Predictions
        _, preds = torch.max(outputs, 1)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    return epoch_loss, epoch_acc.item()

def validate_epoch(model, val_loader, criterion, device):
    """Validasi untuk satu epoch"""
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Predictions
            _, preds = torch.max(outputs, 1)
            
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = running_corrects.double() / len(val_loader.dataset)
    return epoch_loss, epoch_acc.item()

def train_model(model, train_loader, val_loader, criterion, optimizer, 
                scheduler, early_stopping, model_checkpoint, 
                num_epochs=50, device='cpu'):
    """
    Training loop lengkap dengan tracking
    """
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'lr': []
    }
    
    print("=" * 70)
    print(f"Starting training for {num_epochs} epochs...")
    print("=" * 70)
    
    for epoch in range(num_epochs):
        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        history['lr'].append(current_lr)
        
        # Training
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        
        # Validation
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print progress
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
              f"LR: {current_lr:.6f}")
        
        # Scheduler step
        scheduler.step(val_loss)
        
        # Model checkpoint
        model_checkpoint(val_loss, model)
        
        # Early stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"\nEarly stopping triggered at epoch {epoch+1}")
            break
    
    print("=" * 70)
    print("Training completed!")
    print("=" * 70)
    
    return history

# Buat folder model jika belum ada
Path("../model").mkdir(parents=True, exist_ok=True)

## 6. Mulai Training

In [ ]:
# Mulai training
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    early_stopping=early_stopping,
    model_checkpoint=model_checkpoint,
    num_epochs=100,
    device=device
)

Starting training for 100 epochs...


Epoch [1/100] | Train Loss: 0.5583 Acc: 0.7908 | Val Loss: 0.3156 Acc: 0.8956 | LR: 0.001000
Model saved with val_loss: 0.315623


Epoch [2/100] | Train Loss: 0.3937 Acc: 0.8542 | Val Loss: 0.3645 Acc: 0.8608 | LR: 0.001000
EarlyStopping counter: 1/10


Epoch [3/100] | Train Loss: 0.3164 Acc: 0.8675 | Val Loss: 0.5080 Acc: 0.7865 | LR: 0.001000
EarlyStopping counter: 2/10


Epoch [4/100] | Train Loss: 0.3036 Acc: 0.8727 | Val Loss: 0.1794 Acc: 0.9002 | LR: 0.001000
Model saved with val_loss: 0.179360


...

Epoch [36/100] | Train Loss: 0.0359 Acc: 0.9849 | Val Loss: 0.0092 Acc: 0.9977 | LR: 0.000250
Model saved with val_loss: 0.009226


Epoch [37/100] | Train Loss: 0.0402 Acc: 0.9861 | Val Loss: 0.0104 Acc: 0.9977 | LR: 0.000250
EarlyStopping counter: 1/10


Epoch [38/100] | Train Loss: 0.0401 Acc: 0.9884 | Val Loss: 0.0048 Acc: 1.0000 | LR: 0.000250
Model saved with val_loss: 0.004840


Epoch [39/100] | Train Loss: 0.0423 Acc: 0.9866 | Val Loss: 0.0117 Acc: 0.9954 | LR: 0.000250
EarlyStopping counter: 1/10


Training:  94%|█████████▍| 51/54 [01:19<00:04,  1.54s/it]

## 7. Visualisasi Learning Curve dan Akurasi

In [ ]:
def plot_training_history(history):
    """Visualisasi learning curve, akurasi, dan learning rate"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Plot 1: Loss
    axes[0].plot(epochs, history['train_loss'], 'b-o', label='Train Loss', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-o', label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Learning Curve (Loss)', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Accuracy
    axes[1].plot(epochs, history['train_acc'], 'b-o', label='Train Accuracy', linewidth=2)
    axes[1].plot(epochs, history['val_acc'], 'r-o', label='Validation Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title('Learning Curve (Accuracy)', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    # Plot 3: Learning Rate
    axes[2].plot(epochs, history['lr'], 'g-o', linewidth=2)
    axes[2].set_xlabel('Epoch', fontsize=12)
    axes[2].set_ylabel('Learning Rate', fontsize=12)
    axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_yscale('log')
    
    plt.tight_layout()
    plt.savefig('../model/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print summary
    print(f"\n{'='*50}")
    print(f"Training Summary:")
    print(f"{'='*50}")
    print(f"Best Train Loss: {min(history['train_loss']):.4f}")
    print(f"Best Val Loss: {min(history['val_loss']):.4f}")
    print(f"Best Train Accuracy: {max(history['train_acc']):.4f}")
    print(f"Best Val Accuracy: {max(history['val_acc']):.4f}")
    print(f"Final Learning Rate: {history['lr'][-1]:.6f}")
    print(f"Total Epochs: {len(history['train_loss'])}")
    print(f"{'='*50}")

plot_training_history(history)

## 8. Load Model Terbaik dan Evaluasi pada Test Set

In [ ]:
# Load model terbaik
checkpoint = torch.load("../model/best_cnn_model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best model with validation loss: {checkpoint['val_loss']:.4f}")

def evaluate_model(model, test_loader, device):
    """
    Evaluasi model pada test set
    """
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels), np.array(all_probs)

# Evaluasi
predictions, ground_truth, probabilities = evaluate_model(model, test_loader, device)
print(f"\nEvaluated {len(predictions)} images")

## 9. Confusion Matrix dan Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def print_classification_metrics(predictions, ground_truth, class_names):
    """
    Print detailed classification metrics
    """
    print(f"\n{'='*70}")
    print(f"Classification Metrics")
    print(f"{'='*70}")
    
    # Overall accuracy
    accuracy = accuracy_score(ground_truth, predictions)
    print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    # Per-class accuracy
    print(f"\nPer-Class Accuracy:")
    for i, class_name in enumerate(class_names):
        class_mask = ground_truth == i
        class_acc = np.mean(predictions[class_mask] == ground_truth[class_mask])
        print(f"  {class_name}: {class_acc:.4f} ({class_acc*100:.2f}%)")
    
    # Detailed classification report
    print(f"\n{'='*70}")
    print("Detailed Classification Report:")
    print(f"{'='*70}")
    print(classification_report(ground_truth, predictions, target_names=class_names, digits=4))

# Print metrics
class_names = test_dataset.classes
print_classification_metrics(predictions, ground_truth, class_names)

## 10. Visualisasi Confusion Matrix

In [ ]:
def plot_confusion_matrix(predictions, ground_truth, class_names):
    """
    Visualisasi confusion matrix
    """
    cm = confusion_matrix(ground_truth, predictions)
    
    # Normalize confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Raw counts
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[0], cbar_kws={'label': 'Count'})
    axes[0].set_xlabel('Predicted Label', fontsize=12)
    axes[0].set_ylabel('True Label', fontsize=12)
    axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
    
    # Plot 2: Normalized (percentages)
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[1], cbar_kws={'label': 'Percentage'})
    axes[1].set_xlabel('Predicted Label', fontsize=12)
    axes[1].set_ylabel('True Label', fontsize=12)
    axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../model/confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_confusion_matrix(predictions, ground_truth, class_names)

## 11. Visualisasi Prediksi: Benar vs Salah

In [ ]:
def visualize_predictions(test_loader, model, class_names, device, n_correct=5, n_wrong=5):
    """
    Visualisasi prediksi yang benar dan salah
    """
    model.eval()
    
    # Denormalize function
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    def denormalize(img):
        img = img.numpy().transpose((1, 2, 0))
        img = std * img + mean
        img = np.clip(img, 0, 1)
        return img
    
    correct_images = []
    wrong_images = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images_gpu = images.to(device)
            labels_gpu = labels.to(device)
            
            outputs = model(images_gpu)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            for i in range(len(labels)):
                if len(correct_images) >= n_correct and len(wrong_images) >= n_wrong:
                    break
                
                img = images[i].cpu()
                label = labels[i].item()
                pred = preds[i].item()
                prob = probs[i].cpu().numpy()
                
                if pred == label and len(correct_images) < n_correct:
                    correct_images.append((img, label, pred, prob))
                elif pred != label and len(wrong_images) < n_wrong:
                    wrong_images.append((img, label, pred, prob))
            
            if len(correct_images) >= n_correct and len(wrong_images) >= n_wrong:
                break
    
    # Plot correct predictions
    fig, axes = plt.subplots(2, n_correct, figsize=(n_correct * 3, 6))
    
    for i, (img, label, pred, prob) in enumerate(correct_images):
        ax = axes[0, i] if n_correct > 1 else axes[0]
        ax.imshow(denormalize(img))
        ax.axis('off')
        ax.set_title(f'True: {class_names[label]}\nPred: {class_names[pred]}\nConf: {prob[pred]:.2%}',
                    fontsize=10, color='green', fontweight='bold')
    
    if n_correct > 1:
        axes[0, 0].set_ylabel('CORRECT\nPREDICTIONS', fontsize=12, fontweight='bold', color='green')
    
    # Plot wrong predictions
    for i, (img, label, pred, prob) in enumerate(wrong_images):
        ax = axes[1, i] if n_wrong > 1 else axes[1]
        ax.imshow(denormalize(img))
        ax.axis('off')
        ax.set_title(f'True: {class_names[label]}\nPred: {class_names[pred]}\nConf: {prob[pred]:.2%}',
                    fontsize=10, color='red', fontweight='bold')
    
    if n_wrong > 1:
        axes[1, 0].set_ylabel('WRONG\nPREDICTIONS', fontsize=12, fontweight='bold', color='red')
    
    plt.suptitle('Sample Predictions: Correct vs Wrong', fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig('../model/prediction_samples.png', dpi=300, bbox_inches='tight')
    plt.show()

visualize_predictions(test_loader, model, class_names, device, n_correct=5, n_wrong=5)

## 12. Visualisasi Prediksi per Kelas

In [ ]:
def visualize_predictions_per_class(test_loader, model, class_names, device, n_samples=4):
    """
    Visualisasi beberapa prediksi untuk setiap kelas
    """
    model.eval()
    
    # Denormalize function
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    def denormalize(img):
        img = img.numpy().transpose((1, 2, 0))
        img = std * img + mean
        img = np.clip(img, 0, 1)
        return img
    
    # Collect samples for each class
    class_samples = {i: [] for i in range(len(class_names))}
    
    with torch.no_grad():
        for images, labels in test_loader:
            images_gpu = images.to(device)
            
            outputs = model(images_gpu)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            for i in range(len(labels)):
                label = labels[i].item()
                if len(class_samples[label]) < n_samples:
                    img = images[i].cpu()
                    pred = preds[i].item()
                    prob = probs[i].cpu().numpy()
                    class_samples[label].append((img, label, pred, prob))
            
            if all(len(samples) >= n_samples for samples in class_samples.values()):
                break
    
    # Plot
    fig, axes = plt.subplots(len(class_names), n_samples, figsize=(n_samples * 3, len(class_names) * 3))
    
    for class_idx, class_name in enumerate(class_names):
        for i, (img, label, pred, prob) in enumerate(class_samples[class_idx]):
            ax = axes[class_idx, i] if len(class_names) > 1 else axes[i]
            ax.imshow(denormalize(img))
            ax.axis('off')
            
            is_correct = label == pred
            color = 'green' if is_correct else 'red'
            title = f'Pred: {class_names[pred]}\nConf: {prob[pred]:.2%}'
            ax.set_title(title, fontsize=10, color=color, fontweight='bold')
            
            if i == 0:
                ax.set_ylabel(f'{class_name}\n(True Label)', 
                            fontsize=11, fontweight='bold')
    
    plt.suptitle('Sample Predictions per Class', fontsize=14, fontweight='bold', y=0.99)
    plt.tight_layout()
    plt.savefig('../model/predictions_per_class.png', dpi=300, bbox_inches='tight')
    plt.show()

visualize_predictions_per_class(test_loader, model, class_names, device, n_samples=4)

## 13. Ringkasan Akhir dan Kesimpulan

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def print_final_summary(predictions, ground_truth, class_names, history):
    """
    Cetak ringkasan akhir dari seluruh eksperimen
    """
    print("\n" + "="*80)
    print(" " * 25 + "FINAL SUMMARY - CNN PROJECT")
    print("="*80)
    
    print("\n### DATASET ###")
    print(f"Classes: {class_names}")
    print(f"Total test images: {len(predictions)}")
    for i, class_name in enumerate(class_names):
        count = np.sum(ground_truth == i)
        print(f"  - {class_name}: {count} images")
    
    print("\n### TRAINING ###")
    print(f"Total epochs trained: {len(history['train_loss'])}")
    print(f"Best training loss: {min(history['train_loss']):.4f}")
    print(f"Best validation loss: {min(history['val_loss']):.4f}")
    print(f"Best training accuracy: {max(history['train_acc']):.4f}")
    print(f"Best validation accuracy: {max(history['val_acc']):.4f}")
    print(f"Final learning rate: {history['lr'][-1]:.6f}")
    
    print("\n### MODEL ARCHITECTURE ###")
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Model type: CNN (Custom Architecture)")
    
    print("\n### TEST SET PERFORMANCE ###")
    accuracy = accuracy_score(ground_truth, predictions)
    
    # Macro-averaged metrics
    precision = precision_score(ground_truth, predictions, average='macro', zero_division=0)
    recall = recall_score(ground_truth, predictions, average='macro', zero_division=0)
    f1 = f1_score(ground_truth, predictions, average='macro', zero_division=0)
    
    print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Macro Precision: {precision:.4f}")
    print(f"Macro Recall: {recall:.4f}")
    print(f"Macro F1-Score: {f1:.4f}")
    
    print(f"\nPer-Class Performance:")
    for i, class_name in enumerate(class_names):
        class_mask = ground_truth == i
        class_acc = np.mean(predictions[class_mask] == ground_truth[class_mask])
        class_precision = precision_score(ground_truth == i, predictions == i, zero_division=0)
        class_recall = recall_score(ground_truth == i, predictions == i, zero_division=0)
        class_f1 = f1_score(ground_truth == i, predictions == i, zero_division=0)
        
        print(f"  {class_name}:")
        print(f"    Accuracy: {class_acc:.4f} | Precision: {class_precision:.4f} | "
              f"Recall: {class_recall:.4f} | F1: {class_f1:.4f}")
    
    print("\n### OUTPUT FILES ###")
    print("✓ Model: ../model/best_cnn_model.pth")
    print("✓ Training history: ../model/training_history.png")
    print("✓ Confusion matrix: ../model/confusion_matrix.png")
    print("✓ Prediction samples: ../model/prediction_samples.png")
    print("✓ Predictions per class: ../model/predictions_per_class.png")
    
    print("\n" + "="*80)
    print(" " * 30 + "PROJECT COMPLETED!")
    print("="*80 + "\n")

print_final_summary(predictions, ground_truth, class_names, history)

## 14. Fungsi Utilitas untuk Prediksi Gambar Baru

In [ ]:
def predict_single_image(image_path, model, class_names, device):
    """
    Prediksi untuk gambar tunggal
    
    Args:
        image_path: Path ke gambar
        model: Model CNN yang sudah dilatih
        class_names: List nama kelas
        device: Device (cuda/cpu)
    
    Returns:
        Dictionary berisi hasil prediksi
    """
    from PIL import Image
    
    # Load dan preprocess gambar
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Denormalize function untuk visualisasi
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    def denormalize(img):
        img = img.numpy().transpose((1, 2, 0))
        img = std * img + mean
        img = np.clip(img, 0, 1)
        return img
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Prediksi
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        probs = torch.softmax(outputs, dim=1)
        confidence, pred_idx = torch.max(probs, 1)
        
        pred_idx = pred_idx.item()
        confidence = confidence.item()
        all_probs = probs[0].cpu().numpy()
    
    # Visualisasi
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original image (denormalized)
    axes[0].imshow(denormalize(image_tensor.cpu().squeeze()))
    axes[0].set_title('Input Image', fontweight='bold', fontsize=12)
    axes[0].axis('off')
    
    # Prediction probabilities bar chart
    colors = ['green' if i == pred_idx else 'lightgray' for i in range(len(class_names))]
    bars = axes[1].barh(class_names, all_probs, color=colors)
    axes[1].set_xlabel('Probability', fontsize=12)
    axes[1].set_title('Class Probabilities', fontweight='bold', fontsize=12)
    axes[1].set_xlim([0, 1])
    
    # Add percentage labels on bars
    for i, (bar, prob) in enumerate(zip(bars, all_probs)):
        axes[1].text(prob + 0.02, bar.get_y() + bar.get_height()/2, 
                    f'{prob:.2%}', va='center', fontsize=10,
                    fontweight='bold' if i == pred_idx else 'normal')
    
    fig.suptitle(f'Prediction: {class_names[pred_idx]} (Confidence: {confidence:.2%})', 
                 fontsize=14, fontweight='bold', color='green')
    plt.tight_layout()
    plt.show()
    
    return {
        'predicted_class': class_names[pred_idx],
        'predicted_index': pred_idx,
        'confidence': confidence,
        'all_probabilities': {class_names[i]: all_probs[i] for i in range(len(class_names))}
    }

# Contoh penggunaan (uncomment untuk mencoba):
# result = predict_single_image(
#     "../dataset/Potato___healthy/<nama_file>.JPG",
#     model, 
#     class_names,
#     device
# )
# print(result)

print("Fungsi predict_single_image() siap digunakan!")
print("Uncomment kode di atas untuk mencoba prediksi pada gambar tunggal.")

Fungsi predict_single_image() siap digunakan!
Uncomment kode di atas untuk mencoba prediksi pada gambar tunggal.
